- This one is based on the great starting [notebook](https://www.kaggle.com/code/nbroad/transformer-ner-baseline-lb-0-576) by @NICHOLAS BROAD.
- please note: I didn't use the deberta-base model I choose here to finetune, this is just a demo.
- In the postprocessing part of inference, I changed **tokens** to **token_map** in two lines. This will bring a huge improvement. There is the [inference notebook](https://www.kaggle.com/code/takanashihumbert/piidd-deberta-model-starter-inference)

#メモ

In [1]:
"""
メモ
tokenizerで，strideとmax_lenを指定することで，strideの長さのトークン分被るようにして，それぞれmax_lenの長さでシーケンスを分割できる．
これを，documentとどこのtokenに対応するかを覚えておく．これで各1024に対し，予測していく．

#構成
最初にtokenizeでmax_lenの長さにtextをばらす．
"""

'\nメモ\ntokenizerで，strideとmax_lenを指定することで，strideの長さのトークン分被るようにして，それぞれmax_lenの長さでシーケンスを分割できる．\nこれを，documentとどこのtokenに対応するかを覚えておく．これで各1024に対し，予測していく．\n\n#構成\n最初にtokenizeでmax_lenの長さにtextをばらす．\n'

In [2]:
!nvidia-smi

Mon Apr 22 02:38:34 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0              48W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [3]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import runtime

Mounted at /content/drive


# Config

In [4]:
class CFG:
  exp = 'exp042'
  n_folds = 4
  trn_folds = [0,1,2,3]
  max_len = 512
  stride = 128
  model_name = 'microsoft/deberta-v3-large'
  seed=42
  target = [
    'EMAIL', 'ID_NUM', 'NAME_STUDENT', 'PHONE_NUM',
    'STREET_ADDRESS', 'URL_PERSONAL', 'USERNAME'
    ]
  original_target = [
    'B-EMAIL', 'B-ID_NUM', 'B-NAME_STUDENT', 'B-PHONE_NUM',
    'B-STREET_ADDRESS', 'B-URL_PERSONAL', 'B-USERNAME', 'I-ID_NUM',
    'I-NAME_STUDENT', 'I-PHONE_NUM', 'I-STREET_ADDRESS', 'I-URL_PERSONAL'
  ]
  DEBUG=False



In [5]:
# ====================================================
# Directory settings
# ====================================================
import os

OUTPUT_DIR = f'/content/drive/MyDrive/Kaggle/competition/PIIDD/outputs/{CFG.exp}/'

if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

input_path = '/content/drive/MyDrive/Kaggle/competition/PIIDD/input/'

# Import

In [6]:
!pip install seqeval evaluate -q
!pip install sentencepiece

!pip install transformers==4.31.0
os.system('pip install -q tokenizers')
!pip install tokenizers==0.13.3

!pip install accelerate -U

import random
import gc
import json
import argparse
from itertools import chain
from functools import partial

import torch
from transformers import AutoTokenizer, Trainer, TrainingArguments
from transformers import AutoModelForTokenClassification, DataCollatorForTokenClassification
import evaluate
from datasets import Dataset, features, DatasetDict
import pandas as pd
import numpy as np

from sklearn.model_selection import KFold,StratifiedKFold

from scipy.special import softmax
from torch.nn import CrossEntropyLoss
from datasets import load_dataset, concatenate_datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 67.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.0 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existin

# seed

In [7]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(seed=CFG.seed)

In [8]:
reference_data = json.load(open(input_path+"/train.json", "r"))

df = pd.DataFrame(reference_data)[['document', 'tokens', 'labels']].copy()
df = df.explode(['tokens', 'labels']).reset_index(drop=True).rename(columns={'tokens': 'token', 'labels': 'label'})

df['token'] = df.groupby('document').cumcount()

label_list = df['label'].unique().tolist()

reference_df = df[df['label'] != 'O'].copy()

reference_df

,document,token,label
9,7,9,B-NAME_STUDENT
10,7,10,I-NAME_STUDENT
482,7,482,B-NAME_STUDENT
483,7,483,I-NAME_STUDENT
741,7,741,B-NAME_STUDENT
...,...,...,...
2840591,15717,365,B-ID_NUM
2841190,15717,964,B-ID_NUM
3889614,19280,54,B-ID_NUM
3889615,19280,55,I-ID_NUM


In [9]:
# make random predictions on a subset of tokens
rng = random.Random(42)
pred_df = reference_df.sample(frac=0.8).sort_values(by='document').reset_index(drop=True)
pred_df['label'] = pred_df['label'].apply(lambda x: rng.choice(label_list) if rng.random() >= 0.5 else x)
pred_df['row_id'] = list(range(len(pred_df)))
pred_df = pred_df[['row_id', 'document', 'token', 'label']].copy()

pred_df

,row_id,document,token,label
0,0,7,742,O
1,1,7,482,B-URL_PERSONAL
2,2,7,483,I-NAME_STUDENT
3,3,7,9,B-STREET_ADDRESS
4,4,10,0,B-PHONE_NUM
...,...,...,...,...
2186,2186,15717,365,B-ID_NUM
2187,2187,15717,964,I-NAME_STUDENT
2188,2188,19280,54,B-ID_NUM
2189,2189,19280,55,B-EMAIL


In [10]:
from collections import defaultdict
from typing import Dict


class PRFScore:
    """A precision / recall / F score."""

    def __init__(
        self,
        *,
        tp: int = 0,
        fp: int = 0,
        fn: int = 0,
    ) -> None:
        self.tp = tp
        self.fp = fp
        self.fn = fn

    def __len__(self) -> int:
        return self.tp + self.fp + self.fn

    def __iadd__(self, other):  # in-place add
        self.tp += other.tp
        self.fp += other.fp
        self.fn += other.fn
        return self

    def __add__(self, other):
        return PRFScore(
            tp=self.tp + other.tp, fp=self.fp + other.fp, fn=self.fn + other.fn
        )

    def score_set(self, cand: set, gold: set) -> None:
        self.tp += len(cand.intersection(gold))
        self.fp += len(cand - gold)
        self.fn += len(gold - cand)

    @property
    def precision(self) -> float:
        return self.tp / (self.tp + self.fp + 1e-100)

    @property
    def recall(self) -> float:
        return self.tp / (self.tp + self.fn + 1e-100)

    @property
    def f1(self) -> float:
        p = self.precision
        r = self.recall
        return 2 * ((p * r) / (p + r + 1e-100))

    @property
    def f5(self) -> float:
        beta = 5
        p = self.precision
        r = self.recall

        fbeta = (1+(beta**2))*p*r / ((beta**2)*p + r + 1e-100)
        return fbeta

    def to_dict(self) -> Dict[str, float]:
        return {"p": self.precision, "r": self.recall, "f5": self.f5}


def eval_metrics(pred_df, gt_df):
    """
    Compute the LB metric (lb) and other auxiliary metrics
    """

    references = {(row.document, row.token, row.label) for row in gt_df.itertuples()}
    predictions = {(row.document, row.token, row.label) for row in pred_df.itertuples()}

    score_per_type = defaultdict(PRFScore)
    references = set(references)

    for ex in predictions:
        pred_type = ex[-1] # (document, token, label)
        # if pred_type != 'O':
        #     pred_type = pred_type[2:] # avoid B- and I- prefix

        if pred_type not in score_per_type:
            score_per_type[pred_type] = PRFScore()

        if ex in references:
            score_per_type[pred_type].tp += 1
            references.remove(ex)
        else:
            score_per_type[pred_type].fp += 1

    for doc, tok, ref_type in references:
        # if ref_type != 'O':
        #     ref_type = ref_type[2:] # avoid B- and I- prefix

        if ref_type not in score_per_type:
            score_per_type[ref_type] = PRFScore()
        score_per_type[ref_type].fn += 1

    totals = PRFScore()

    for prf in score_per_type.values():
        totals += prf

    return {
        "ents_p": totals.precision,
        "ents_r": totals.recall,
        "ents_f5": totals.f5,
        "ents_per_type": {k: v.to_dict() for k, v in score_per_type.items() if k!= 'O'},
    }

In [11]:
eval_metrics(pred_df.copy(), reference_df.copy())

{'ents_p': 0.5371976266544957,
 'ents_r': 0.42971887550200805,
 'ents_f5': 0.4330512552005208,
 'ents_per_type': {'B-NAME_STUDENT': {'p': 0.9256329113924051,
   'r': 0.42857142857142855,
   'f5': 0.4376096901343614},
  'I-NAME_STUDENT': {'p': 0.8971428571428571,
   'r': 0.42974452554744524,
   'f5': 0.4385317815577439},
  'B-STREET_ADDRESS': {'p': 0.025974025974025976,
   'r': 1.0,
   'f5': 0.4094488188976378},
  'B-URL_PERSONAL': {'p': 0.41509433962264153,
   'r': 0.4,
   'f5': 0.4005602240896358},
  'B-ID_NUM': {'p': 0.2857142857142857,
   'r': 0.4358974358974359,
   'f5': 0.42725954567423874},
  'I-PHONE_NUM': {'p': 0.06593406593406594,
   'r': 0.4,
   'f5': 0.33476394849785407},
  'I-URL_PERSONAL': {'p': 0.0136986301369863,
   'r': 1.0,
   'f5': 0.26530612244897955},
  'I-STREET_ADDRESS': {'p': 0.10576923076923077,
   'r': 0.55,
   'f5': 0.4735099337748345},
  'B-USERNAME': {'p': 0.03773584905660377,
   'r': 0.6666666666666666,
   'f5': 0.40624999999999994},
  'B-EMAIL': {'p': 0.16

# Cross Validation


In [12]:
def creates_folds(data,num_splits):
  '''
  Parameters:
  - data: (json) This is only piidd dataset.
  - num_splits: (int)

  Returns:
  - data: (json)
  '''
  labels = [item['is_label'] for item in data]
  kf = StratifiedKFold(n_splits=num_splits, shuffle=True, random_state=CFG.seed)
  cnt=[0]*(num_splits)
  for i,(_t, _v) in  enumerate(kf.split(data,labels)):
    for idx in _v:
      data[idx]['fold']=i
  return data


data = json.load(open(input_path+"train.json"))

for i in range(len(data)):
  flag=True
  for label in data[i]['labels']:
      if label in CFG.target:
          data[i]['is_label']=1
          flag=False
          break
  if flag:
    data[i]['is_label']=0

data=creates_folds(data,CFG.n_folds)


external_data = json.load(open(input_path+"mixtral-8x7b-v1.json"))
print(len(external_data))
print(external_data[0].keys())

for i in range(len(external_data)):
  external_data[i]['fold']=-1

data = data+external_data

for i in range(len(data)):
  data[i]['document']=i+1

# del external_data1,external_data
gc.collect()

print(len(data))
print(data[0].keys())


2355
dict_keys(['document', 'full_text', 'tokens', 'labels', 'trailing_whitespace'])
9162
dict_keys(['document', 'full_text', 'tokens', 'trailing_whitespace', 'labels', 'is_label', 'fold'])


In [13]:
df = pd.DataFrame(data)[['document', 'tokens', 'labels','fold']].copy()
df = df.explode(['tokens', 'labels']).reset_index(drop=True).rename(columns={'tokens': 'token', 'labels': 'label'})

df['token'] = df.groupby('document').cumcount()

label_list = df['label'].unique().tolist()

reference_df = df[(df['label'] != 'O')&(df['fold']!=-1)].copy()

In [14]:
reference_df

,document,token,label,fold
9,1,9,B-NAME_STUDENT,2
10,1,10,I-NAME_STUDENT,2
482,1,482,B-NAME_STUDENT,2
483,1,483,I-NAME_STUDENT,2
741,1,741,B-NAME_STUDENT,2
...,...,...,...,...
2840591,3770,365,B-ID_NUM,3
2841190,3770,964,B-ID_NUM,3
3889614,5240,54,B-ID_NUM,1
3889615,5240,55,I-ID_NUM,1


In [15]:
for i in range(len(data)):
  for idx,labels in enumerate(data[i]['labels']):
    if labels in CFG.original_target:
      data[i]['labels'][idx]=data[i]['labels'][idx][2:]

# id2label

In [16]:
all_labels = [
    'EMAIL', 'ID_NUM', 'NAME_STUDENT', 'PHONE_NUM',
    'STREET_ADDRESS', 'URL_PERSONAL', 'USERNAME','O'
    ]
label2id = {l: i for i,l in enumerate(all_labels)}
id2label = {v:k for k,v in label2id.items()}

print(id2label)

{0: 'EMAIL', 1: 'ID_NUM', 2: 'NAME_STUDENT', 3: 'PHONE_NUM', 4: 'STREET_ADDRESS', 5: 'URL_PERSONAL', 6: 'USERNAME', 7: 'O'}


In [17]:
id2label

{0: 'EMAIL',
 1: 'ID_NUM',
 2: 'NAME_STUDENT',
 3: 'PHONE_NUM',
 4: 'STREET_ADDRESS',
 5: 'URL_PERSONAL',
 6: 'USERNAME',
 7: 'O'}

In [18]:
def is_nested_list(lst):
    return any(isinstance(item, list) for item in lst)

In [19]:
def tokenize(example, tokenizer, label2id):
    text = []
    token_map = []
    # these are at the character level
    labels = []
    targets = []
    idx=0

    for t, l, ws in zip(example["tokens"], example["provided_labels"], example["trailing_whitespace"]):
        text.append(t)
        #文字レベルでlabelを振っている．
        labels.extend([l]*len(t))
        token_map.extend([idx]*len(t))

        if l in CFG.target:
            targets.append(1)
        else:
            targets.append(0)
        # if there is trailing whitespace
        if ws:
            text.append(" ")
            labels.append("O")
            token_map.append(-1)
        idx += 1

    target_num = sum(targets)
    labels = np.array(labels)

    tokenized = tokenizer("".join(text), return_offsets_mapping=True, truncation=True, max_length=CFG.max_len, stride=CFG.stride,return_overflowing_tokens=True)
    text = "".join(text)

    token_samples=[]

    if not is_nested_list(tokenized.offset_mapping):
      token_labels = []
      for start_idx, end_idx in tokenized.offset_mapping:

          # CLS token
          if start_idx == 0 and end_idx == 0:
              token_labels.append(label2id["O"])
              continue

          # case when token starts with whitespace
          if text[start_idx].isspace() and text[start_idx]!='\n' and text[start_idx]!='\n\n':
              start_idx += 1

          token_labels.append(label2id[labels[start_idx]])

      length = len(tokenized.input_ids)

      return [
          tokenized.input_ids,
          tokenized.token_type_ids,
          tokenized.attention_mask,
          tokenized.offset_mapping,
          tokenized.overflow_to_sample_mapping,
          token_labels,
          length,
          target_num,
          1 if target_num>0 else 0,
          example['document'],
          example['fold'],
          token_map,
          example["tokens"]
      ]

    else:
      for i in range(len(tokenized.offset_mapping)):
        token_labels = []

        for start_idx, end_idx in tokenized.offset_mapping[i]:

            # CLS token
            if start_idx == 0 and end_idx == 0:
                token_labels.append(label2id["O"])
                continue

            # case when token starts with whitespace
            if text[start_idx].isspace() and text[start_idx]!='\n' and text[start_idx]!='\n\n':
                start_idx += 1

            token_labels.append(label2id[labels[start_idx]])

        length = len(tokenized.input_ids[i])

        token_sample= [
          tokenized.input_ids[i],
          tokenized.token_type_ids[i],
          tokenized.attention_mask[i],
          tokenized.offset_mapping[i],
          tokenized.overflow_to_sample_mapping[i],
          token_labels,
          length,
          target_num,
          1 if target_num>0 else 0,
          example['document'],
          example['fold'],
          token_map,
          example["tokens"]
      ]
        token_samples.append(token_sample)

      return token_samples

## competition metrics

In [20]:
from seqeval.metrics import recall_score, precision_score
from seqeval.metrics import classification_report
from seqeval.metrics import f1_score
def rule_base(pred_df):

    #URL
    pred_df=pred_df[~((pred_df['label'].str.contains('URL_PERSONAL')) & (~pred_df['token_str'].str.contains('://')))]

    #EMAIL
    pred_df=pred_df[~((pred_df['label'].str.contains('EMAIL')) & (~pred_df['token_str'].str.match(r'^[^@]+@[^@]+\.com$')))]

    import re
    # 1. 1文字または2文字の場合を省く
    pred_df = pred_df[~((pred_df['label'].str.contains('ID_NUM')) & (pred_df['token_str'].str.len() <= 3))]

    # 2. 5文字以上の連続した数字を含まない場合を省く
    pred_df = pred_df[~((pred_df['label'].str.contains('ID_NUM')) & (~pred_df['token_str'].apply(lambda x: bool(re.search(r'\d{3,}', x)))))]
    pred_df = pred_df[~((pred_df['label'].str.contains('ID_NUM')) & (pred_df['token_str'].str.contains('://')) & ((pred_df['token_str'].str.contains('http'))))]

    # USER-NAMEを含み、かつ5文字以上でない行を省く
    pred_df = pred_df[~((pred_df['label'].str.contains('USER_NAME')) & (pred_df['token_str'].str.len() < 4))]

    # 1. 1文字の場合を省く
    pred_df = pred_df[~((pred_df['label'].str.contains('NAME_STUDENT')) & (pred_df['token_str'].str.len() == 1))]

    # 2. 一文字目は大文字で、それ以降は小文字であることを確認する
    pred_df = pred_df[~((pred_df['label'].str.contains('NAME_STUDENT')) & (~pred_df['token_str'].str.match(r'^[A-Z][a-z]+$')))]

    pred_df = pred_df[~((pred_df['label'].str.contains('NAME_STUDENT')) & (pred_df['token_str']=='Dr'))]
    return pred_df



def parse_predictions(predictions, id2label, ds,fold,threshold=0.9):

    pred_softmax = np.exp(predictions) / np.sum(np.exp(predictions), axis = 2).reshape(predictions.shape[0],predictions.shape[1],1)
    triplets = []
    row, document, token, labels, token_str = [], [], [], [], []
    for i, (p, token_map, offsets, tokens, doc) in enumerate(zip(pred_softmax, ds["token_map"], ds["offset_mapping"], ds["tokens"], ds["document"])):

        for token_pred, (start_idx, end_idx) in zip(p, offsets):
            # label_pred = id2label[token_pred]

            if start_idx + end_idx == 0: continue

            if token_map[start_idx] == -1:
                start_idx += 1

            # ignore "\n\n"
            while start_idx < len(token_map) and tokens[token_map[start_idx]].isspace() and tokens[token_map[start_idx]]!='\n' and tokens[token_map[start_idx]]!='\n\n':
                start_idx += 1

            if start_idx >= len(token_map): break

            token_id = token_map[start_idx]

            # ignore "O" predictions and whitespace preds
            if token_id != -1:
                #ここを改善

                row.append(i)
                document.append(doc)
                token.append(token_id)
                labels.append(token_pred)
                token_str.append(tokens[token_id])

    df = pd.DataFrame({
        "eval_row": row,
        "document": document,
        "token": token,
        "label": labels,
        "token_str": token_str
    })

    print(id2label)
    d={}
    for i in range(8):
        d[i]=[]

    for p in labels:
        for i in range(8):
            d[i].append(p[i])

    for i in range(8):
        df[id2label[i]]=d[i]

    df=df[df['token_str']!='\n\n']


    df=df.groupby(['document', 'token', 'token_str']).mean().reset_index()



    # 'O'以外の列で最大値を持つ列名を'label'列に書き込む関数
    def max_column_name(row):
        return row.idxmax()

    # 'O'以外の列で最大値を持つ列名を'label'列に書き込む
    df.to_csv(OUTPUT_DIR+f'preds_{threshold}_{fold}_p.csv',index=False)
    df.loc[:, 'label'] = df[CFG.target].apply(max_column_name, axis=1)
    df=rule_base(df)
    df=df[df['O'] < threshold]

    df=df.drop(columns=CFG.target+['O'])

    df = df.sort_values(by=['document', 'token'])

    # 初期化
    prev_doc = None
    prev_token = None
    prev_label = None

    # 'label' 列の処理
    for index, row in df.iterrows():
      if row['document'] == prev_doc and row['token'] == prev_token + 1 and row['label'] == prev_label:
        # 同じ document かつ連続している token の場合、I- を付ける
        df.loc[index, 'label'] = 'I-' + row['label']
      else:
        # 初めての document または非連続な token の場合、B- を付ける
        df.loc[index, 'label'] = 'B-' + row['label']
      # 前の行の情報を更新
      prev_doc = row['document']
      prev_token = row['token']
      prev_label = row['label']
    df=df[['document', 'token', 'label']]

    df["row_id"] = list(range(len(df)))
    df.to_csv(OUTPUT_DIR+f'preds_{threshold}_{fold}.csv',index=False)
    display(df)
    return df


def compute_metrics(p, id2label, valid_ds, valid_df,fold,threshold=0.97):
    """
    Compute the LB metric (lb) and other auxiliary metrics
    """
    predictions, labels = p

    pred_df = parse_predictions(predictions, id2label, valid_ds,fold,threshold=threshold)

    references = {(row.document, row.token, row.label) for row in valid_df.itertuples()}
    predictions = {(row.document, row.token, row.label) for row in pred_df.itertuples()}

    score_per_type = defaultdict(PRFScore)
    references = set(references)

    for ex in predictions:
        pred_type = ex[-1] # (document, token, label)
        # if pred_type != 'O':
        #     pred_type = pred_type[2:] # avoid B- and I- prefix

        if pred_type not in score_per_type:
            score_per_type[pred_type] = PRFScore()

        if ex in references:
            score_per_type[pred_type].tp += 1
            references.remove(ex)
        else:
            score_per_type[pred_type].fp += 1

    for doc, tok, ref_type in references:
        # if ref_type != 'O':
        #     ref_type = ref_type[2:] # avoid B- and I- prefix

        if ref_type not in score_per_type:
            score_per_type[ref_type] = PRFScore()
        score_per_type[ref_type].fn += 1

    totals = PRFScore()

    for prf in score_per_type.values():
        totals += prf


    results = {
        "ents_p": totals.precision,
        "ents_r": totals.recall,
        "ents_f5": totals.f5,
    }
    return results

    # results = {
    #     "ents_p": totals.precision,
    #     "ents_r": totals.recall,
    #     "ents_f5": totals.f5,
    #     "ents_per_type": {k: v.to_dict() for k, v in score_per_type.items() if k!= 'O'},
    # }

    # # Unpack nested dictionaries
    # final_results = {}
    # for key, value in results.items():
    #     if isinstance(value, dict):
    #         for n, v in value.items():
    #             if isinstance(v, dict):
    #                 for n2, v2 in v.items():
    #                     final_results[f"{key}_{n}_{n2}"] = v2
    #             else:
    #                 final_results[f"{key}_{n}"] = v
    #     else:
    #         final_results[key] = value

    # return final_results

In [21]:
def freeze_layer(model,FREEZE_LAYERS = 6,FREEZE_EMBEDDINGS = False):
  if FREEZE_EMBEDDINGS:
      print('Freezing embeddings.')
      for param in model.deberta.embeddings.parameters():
          param.requires_grad = False

  if FREEZE_LAYERS>0:
      print(f'Freezing {FREEZE_LAYERS} layers.')
      for layer in model.deberta.encoder.layer[:FREEZE_LAYERS]:
          for param in layer.parameters():
              param.requires_grad = False


# Training Arguments

In [22]:
args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    fp16=True,
    learning_rate=2e-5,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    report_to="none",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    overwrite_output_dir=True,
    load_best_model_at_end=True,
    lr_scheduler_type='cosine',
    metric_for_best_model="ents_f5",
    greater_is_better=True,
    weight_decay=0.01
)


In [23]:
from transformers import DebertaV2PreTrainedModel, DebertaV2Model
from transformers.modeling_outputs import TokenClassifierOutput
import torch.nn as nn
class LSTMHead(nn.Module):
    def __init__(self, in_features, hidden_dim, n_layers):
        super().__init__()
        self.lstm = nn.LSTM(in_features,
                            hidden_dim,
                            n_layers,
                            batch_first=True,
                            bidirectional=True,
                            dropout=0.1)
        self.out_features = hidden_dim

    def forward(self, x):
        self.lstm.flatten_parameters()
        hidden, (_, _) = self.lstm(x)
        out = hidden
        return out


# Copied from transformers.models.deberta.modeling_deberta.DebertaForTokenClassification with Deberta->DebertaV2
class CustomModel(DebertaV2PreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels

        self.deberta = DebertaV2Model(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        self.head = LSTMHead(in_features=config.hidden_size, hidden_dim=config.hidden_size//2, n_layers=1)
        # Initialize weights and apply final processing
        self.post_init()

    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, labels=None, **kwargs):

        outputs = self.deberta(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )

        sequence_output = outputs.last_hidden_state
        sequence_output = self.dropout(sequence_output)
        sequence_output = self.head(sequence_output)
        logits = self.classifier(sequence_output)

        loss = None
        if labels is not None:
            loss_fct = CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        return TokenClassifierOutput(
            loss=loss, logits=logits, hidden_states=outputs.hidden_states, attentions=outputs.attentions
        )

class CustomTrainer(Trainer):
    def __init__(self, *args, class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        # Assuming class_weights is a Tensor of weights for each class
        self.class_weights = class_weights

    def compute_loss(self, model, inputs, return_outputs=False):
        # Extract labels
        labels = inputs.pop("labels")
        # Forward pass
        outputs = model(**inputs)
        logits = outputs.logits
        # Reshape for loss calculation
        loss_fct = CrossEntropyLoss(weight=self.class_weights)
        if self.label_smoother is not None and "labels" in inputs:
            loss = self.label_smoother(outputs, inputs)
        else:
            loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))

        return (loss, outputs) if return_outputs else loss

## training

In [24]:
# if __name__ == "__main__":

#   if CFG.DEBUG:
#     data=data[0:200]
#     args.num_train_epochs = 1
#     CFG.max_len=256
#     CFG.stride=5
#     CFG.trn_folds=[0]

#   tokenizer = AutoTokenizer.from_pretrained(CFG.model_name)

#   ds = Dataset.from_dict({
#       "full_text": [x["full_text"] for x in data],
#       "document": [x["document"] for x in data],
#       "tokens": [x["tokens"] for x in data],
#       "trailing_whitespace": [x["trailing_whitespace"] for x in data],
#       "provided_labels": [x["labels"] for x in data],
#       "fold": [x["fold"] for x in data]
#   })




#   data_list=[]
#   for example in ds:
#       # 各サンプルに対してtokenize関数を呼び出し
#       dataset_list=tokenize(example, tokenizer=tokenizer, label2id=label2id)
#       for d in dataset_list:
#         data_list.append(d)


#   ds=Dataset.from_dict({
#     'input_ids': [x[0] for x in data_list],
#     'token_type_ids': [x[1] for x in data_list],
#     "attention_mask":[x[2] for x in data_list],
#     'offset_mapping': [x[3] for x in data_list],
#     'overflow_to_sample_mapping':[x[4] for x in data_list],
#     "labels": [x[5] for x in data_list],
#     "length": [x[6] for x in data_list],
#     "target_num": [x[7] for x in data_list],
#     "group": [x[8] for x in data_list],
#     'document':[x[9] for x in data_list],
#     'fold':[x[10] for x in data_list],
#     'token_map':[x[11] for x in data_list],
#     'tokens':[x[12] for x in data_list]
#     })

#   # ds = ds.map(tokenize, fn_kwargs={"tokenizer": tokenizer, "label2id": label2id}, num_proc=2)

#   ds = ds.class_encode_column("group")
#   collator = DataCollatorForTokenClassification(tokenizer, pad_to_multiple_of=16)
#   oof = []
#   labels = []
#   for fold in CFG.trn_folds:
#     ds_i = DatasetDict(
#         {
#             'train': ds.filter(lambda x: x['fold']!=fold),
#             'eval': ds.filter(lambda x: x['fold']==fold),
#         }
#       )
#     ds_i = ds_i.remove_columns(['fold'])
#     reference_df_i=reference_df[reference_df['fold']==fold]
#     args.output_dir = OUTPUT_DIR + f'fold_{fold}/'

#     reference_df_i.to_csv(OUTPUT_DIR+f'reference_{fold}.csv',index=False)

#     # runtime.unassign()

#     # train
#     model = CustomModel.from_pretrained(
#         CFG.model_name,
#         num_labels=len(all_labels),
#         id2label=id2label,
#         label2id=label2id,
#         ignore_mismatched_sizes=True
#     )
#     # freeze_layer(model)

#     trainer = Trainer(
#         model=model,
#         args=args,
#         train_dataset=ds_i['train'],
#         eval_dataset=ds_i['eval'],
#         data_collator=collator,
#         tokenizer=tokenizer,
#         compute_metrics=partial(compute_metrics, id2label=id2label, valid_ds=ds_i['eval'], valid_df=reference_df_i, threshold=0.97),
#     )

#     trainer.train()

#     # eval
#     # Make predictions on the validation dataset
#     preds = trainer.predict(ds_i['eval'])

#     # Compute the final metrics and log them to Weights & Biases
#     print('Computing final metrics...')
#     final_metrics = {
#         f'final_f5_at_{threshold}': compute_metrics((preds.predictions, None), id2label, ds_i['eval'], reference_df_i, threshold=threshold)['ents_f5']
#         for threshold in [0.8,0.9,0.93,0.95,0.96,0.97,0.98,0.99]
#     }
#     print(f'fold-{fold}')
#     print(final_metrics)

#     del model,trainer
#     torch.cuda.empty_cache()
#     gc.collect()




In [ ]:
if __name__ == "__main__":

  if CFG.DEBUG:
    data=data[0:150]
    args.num_train_epochs = 1
    CFG.max_len=128
    CFG.stride=2
    CFG.trn_folds=[0,1,2]

  tokenizer = AutoTokenizer.from_pretrained(CFG.model_name)
  tokenizer.add_tokens(['\n','\n\n'], special_tokens=True)

  ds = Dataset.from_dict({
      "full_text": [x["full_text"] for x in data],
      "document": [x["document"] for x in data],
      "tokens": [x["tokens"] for x in data],
      "trailing_whitespace": [x["trailing_whitespace"] for x in data],
      "provided_labels": [x["labels"] for x in data],
      "fold": [x["fold"] for x in data]
  })




  data_list=[]
  for example in ds:
      # 各サンプルに対してtokenize関数を呼び出し
      dataset_list=tokenize(example, tokenizer=tokenizer, label2id=label2id)
      for d in dataset_list:
        data_list.append(d)


  ds=Dataset.from_dict({
    'input_ids': [x[0] for x in data_list],
    'token_type_ids': [x[1] for x in data_list],
    "attention_mask":[x[2] for x in data_list],
    'offset_mapping': [x[3] for x in data_list],
    'overflow_to_sample_mapping':[x[4] for x in data_list],
    "labels": [x[5] for x in data_list],
    "length": [x[6] for x in data_list],
    "target_num": [x[7] for x in data_list],
    "group": [x[8] for x in data_list],
    'document':[x[9] for x in data_list],
    'fold':[x[10] for x in data_list],
    'token_map':[x[11] for x in data_list],
    'tokens':[x[12] for x in data_list]
    })

  # ds = ds.map(tokenize, fn_kwargs={"tokenizer": tokenizer, "label2id": label2id}, num_proc=2)

  ds = ds.class_encode_column("group")
  collator = DataCollatorForTokenClassification(tokenizer, pad_to_multiple_of=16)
  oof = []
  labels = []
  ds_list=[]
  for fold in CFG.trn_folds:
    ds_i = DatasetDict(
        {
            'train': ds.filter(lambda x: x['fold']!=fold),
            'eval': ds.filter(lambda x: x['fold']==fold),
        }
      )
    ds_list.append(ds_i['eval'])
    ds_i = ds_i.remove_columns(['fold'])
    reference_df_i=reference_df[reference_df['fold']==fold]
    args.output_dir = OUTPUT_DIR + f'fold_{fold}/'

    # train
    model = CustomModel.from_pretrained(
        CFG.model_name,
        num_labels=len(all_labels),
        id2label=id2label,
        label2id=label2id,
        ignore_mismatched_sizes=True
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=ds_i['train'],
        eval_dataset=ds_i['eval'],
        data_collator=collator,
        tokenizer=tokenizer,
        compute_metrics=partial(compute_metrics, id2label=id2label, valid_ds=ds_i['eval'], valid_df=reference_df_i, fold=fold,threshold=0.97)
    )

    trainer.train()

    # eval
    # Make predictions on the validation dataset
    oof_i = trainer.predict(ds_i['eval'])
    oof.append(oof_i.predictions)

    del model,trainer
    torch.cuda.empty_cache()
    gc.collect()

  oof = np.vstack(oof)
  ds_combined=concatenate_datasets(ds_list)
  # Compute the final metrics and log them to Weights & Biases
  print('Computing final metrics...')
  best_f5 = 0.0
  for threshold in [ 0.8, 0.85, 0.9, 0.95, 0.97,0.98,0.99]:
    print(f'threshold:{threshold}')
    score=compute_metrics((oof,None), id2label, ds_combined, reference_df, fold=-1 ,threshold=threshold)
    f5=score['ents_f5']
    print(f'f5:{f5}, threshold:{threshold}')
    if best_f5 < f5:
      best_f5 = f5
      best_threshold = threshold
  print(f'best_f5:{best_f5}, best_threshold:{best_threshold}')




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/580 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:470: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

f5:0.963056371598027, threshold:0.95
threshold:0.97
{0: 'EMAIL', 1: 'ID_NUM', 2: 'NAME_STUDENT', 3: 'PHONE_NUM', 4: 'STREET_ADDRESS', 5: 'URL_PERSONAL', 6: 'USERNAME', 7: 'O'}


,document,token,label,row_id
9,1,9,B-NAME_STUDENT,0
10,1,10,I-NAME_STUDENT,1
437,1,482,B-NAME_STUDENT,2
438,1,483,I-NAME_STUDENT,3
680,1,741,B-NAME_STUDENT,4
...,...,...,...,...
4533934,6694,474,B-NAME_STUDENT,3862
4533935,6694,475,I-NAME_STUDENT,3863
4534184,6694,743,B-NAME_STUDENT,3864
4534185,6694,744,I-NAME_STUDENT,3865


f5:0.9664372010726824, threshold:0.97
threshold:0.98
{0: 'EMAIL', 1: 'ID_NUM', 2: 'NAME_STUDENT', 3: 'PHONE_NUM', 4: 'STREET_ADDRESS', 5: 'URL_PERSONAL', 6: 'USERNAME', 7: 'O'}


,document,token,label,row_id
9,1,9,B-NAME_STUDENT,0
10,1,10,I-NAME_STUDENT,1
437,1,482,B-NAME_STUDENT,2
438,1,483,I-NAME_STUDENT,3
680,1,741,B-NAME_STUDENT,4
...,...,...,...,...
4537815,6700,164,B-NAME_STUDENT,4010
4570416,6755,356,B-NAME_STUDENT,4011
4570433,6755,375,B-NAME_STUDENT,4012
4595542,6791,113,B-NAME_STUDENT,4013


f5:0.965898744654435, threshold:0.98
threshold:0.99
{0: 'EMAIL', 1: 'ID_NUM', 2: 'NAME_STUDENT', 3: 'PHONE_NUM', 4: 'STREET_ADDRESS', 5: 'URL_PERSONAL', 6: 'USERNAME', 7: 'O'}


In [ ]:
#0.002800	0.001133	0.679960	0.978479	0.962231 lstm+freeze(1024,256)
#0.001100 0.000967  0.680872  0.985653  0.968970(f5:0.970 threshold:0.93) lstm(512,128)

In [ ]:
runtime.unassign()

In [ ]:
# '[SPACE]'.isspace()